In [65]:
import sys
import numpy as np
import pandas as pd
sys.path.append("../code/")

In [3]:
from dataloader import APPLIANCE_ORDER, get_train_test_tensor
from ddsc import SparseCoding, reshape_for_sc

In [3]:
# tensor = np.load('../2015-5appliances-true-agg.npy')
ag_tensor = np.load('../2015-5appliances.numpy.npy')
true_tensor = np.load("../2015-5appliances-true-agg.npy")
from sklearn.metrics import mean_absolute_error

In [4]:
ag_agg = ag_tensor[:, 0, :, :]
true_agg = true_tensor[:, 0, :, :]
residual_agg = true_agg - ag_agg

In [5]:
residual_agg = residual_agg.reshape(68, -1)

In [6]:
residual_agg.shape

(68, 2688)

In [7]:
from sklearn.cluster import KMeans

In [22]:
kmeans = KMeans(n_clusters=3, random_state=0).fit(residual_agg)
kmeans.labels_

array([1, 1, 2, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 2,
       0, 1, 1, 0, 1, 0, 2, 1, 1, 1, 1, 0, 1, 1, 0, 1, 2, 1, 1, 1, 1, 0, 2,
       2, 0, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 0, 1], dtype=int32)

In [23]:
cluster = {}
for i in range(3):
    cluster[i] = [x for x in range(68) if kmeans.labels_[x] == i]

In [24]:
print(len(cluster[0]))
print(len(cluster[1]))
print(len(cluster[2]))

12
48
8


In [25]:
true_agg[cluster[0]].shape

(12, 112, 24)

In [27]:
true_tensor[cluster[2]]

array([[[[  3.35300000e+03,   2.86085010e+03,   2.63469995e+03, ...,
            7.18770020e+03,   8.35561621e+03,   7.65276660e+03],
         [  4.06825000e+03,   2.91296655e+03,   2.52794995e+03, ...,
            5.13088330e+03,   4.99460010e+03,   4.23000000e+03],
         [  3.37019995e+03,   3.39739990e+03,   2.83283325e+03, ...,
            7.49095020e+03,   6.99366650e+03,   7.04421680e+03],
         ..., 
         [  2.60933325e+03,   2.21941675e+03,   2.00898328e+03, ...,
            8.04920020e+03,   6.42971680e+03,   6.13303320e+03],
         [  3.36691675e+03,   2.13035010e+03,   1.62875000e+03, ...,
            4.09818311e+03,   2.81793335e+03,   2.55913330e+03],
         [  2.11541675e+03,   1.59543335e+03,   2.04159998e+03, ...,
            5.31328320e+03,   3.03714990e+03,   3.38833325e+03]],

        [[  1.54041663e+03,   1.40081665e+03,   1.17111670e+03, ...,
            3.04471655e+03,   3.01106665e+03,   2.50201660e+03],
         [  1.83594995e+03,   1.28993335e+03,

In [14]:
np.save("../2015-5appliances-true-agg-c1.npy", true_tensor[cluster[0]])
np.save("../2015-5appliances-true-agg-c2.npy", true_tensor[cluster[1]])
np.save("../2015-5appliances-true-agg-c3.npy", true_tensor[cluster[2]])

In [20]:
def mean_predictions(tensor):
    out = []
    for cur_fold in range(5):
        train, test = get_train_test_tensor(tensor, num_folds=num_folds, fold_num=cur_fold)
        pred = np.zeros_like(test[:, 1:, :, :])
#         print(pred.shape)
        
        for appliance in range(test.shape[1]-1):
            for day in range(test.shape[2]):
                pred[:, appliance, day, :] = train[:, appliance+1, day, :].mean(axis=0)
        out.append(pred)
    return np.concatenate(out)

In [34]:
from sklearn.model_selection import KFold
error_mean = {}
num_folds = 5
pred = {}
gt = {}
for j in range(3):
    error_mean[j] = {}
    gt[j] = true_tensor[cluster[j], 1:, :, :]
    pred[j] = mean_predictions(true_tensor[cluster[j]])
    pred[j] = np.minimum(pred[j], true_tensor[cluster[j], 0:1, :,:])

    error_mean[j] = {APPLIANCE_ORDER[i+1]:mean_absolute_error(pred[j][:, i,:,:].flatten(), 
                                                                       gt[j][:, i, :, :].flatten()) for i in range(pred[j].shape[1])}

(12, 6, 112, 24)
(12, 6, 112, 24)
(12, 6, 112, 24)
(12, 6, 112, 24)
(12, 6, 112, 24)
(48, 6, 112, 24)
(48, 6, 112, 24)
(48, 6, 112, 24)
(48, 6, 112, 24)
(48, 6, 112, 24)
(8, 6, 112, 24)
(8, 6, 112, 24)
(8, 6, 112, 24)
(8, 6, 112, 24)
(8, 6, 112, 24)


In [37]:
np.save("../code/baseline/mean-gt.npy", gt)
np.save("../code/baseline/mean-pred.npy", pred)

In [67]:
pd.DataFrame(error_mean)

,0,1,2
dr,138.298335,108.971725,169.691443
dw,34.071017,24.014139,49.025959
fridge,35.831903,29.289916,29.550734
hvac,598.736930,390.463153,785.263499
mw,14.462326,8.981314,24.663397


In [69]:
for i in [0, 1 ,2]:
    print(pd.Series(error_mean[i]))

dr        138.298335
dw         34.071017
fridge     35.831903
hvac      598.736930
mw         14.462326
dtype: float64
dr        108.971725
dw         24.014139
fridge     29.289916
hvac      390.463153
mw          8.981314
dtype: float64
dr        169.691443
dw         49.025959
fridge     29.550734
hvac      785.263499
mw         24.663397
dtype: float64


In [ ]:
dataset = 2
err_mean ={}
gt = tensor[:, 1:, :, :]
pred = mean_predictions(dataset)
# Clamping prediction to aggregate
pred = np.minimum(pred, tensor[:, 0:1, :,:])
err_mean = {APPLIANCE_ORDER[i+1]:mean_absolute_error(pred[:, i,:,:].flatten(), 
                                                                       gt[:, i, :, :].flatten()) for i in range(pred.shape[1])}

# sc without discriminative

In [66]:
dataset = 2
r = {}
mean_r = {}
for cluster in [1, 2 ,3]:
    r[cluster] = {}
    mean_r[cluster] = {}
    for cur_fold in range(5):
        r[cluster][cur_fold] = {}
        mean_r[cluster][cur_fold] = {}
        for num_latent in range(1, 51):
            r[cluster][cur_fold][num_latent] = np.load("../code/baseline/sc-non-nested-cluster/sc-non-error-{}-{}-{}-{}.npy".format(dataset, cluster, cur_fold, num_latent)).item()
            mean_r[cluster][cur_fold][num_latent] = pd.Series(r[cluster][cur_fold][num_latent]).mean()

In [67]:
pd.Series(mean_r[1][0]).idxmin()

1

In [68]:
best_result = {}
best_param = {}
best_mean = {}
for cluster in [1, 2, 3]:
    best_result[cluster] = {}
    best_mean[cluster] = {}
    best_param[cluster] = {}
    for cur_fold in range(5):
        best_result[cluster][cur_fold] = {} 

In [69]:
for cluster in [1, 2, 3]:
    for cur_fold in range(5):
        best_param[cluster][cur_fold] = pd.Series(mean_r[cluster][cur_fold]).idxmin()
        best_result[cluster][cur_fold] = r[cluster][cur_fold][best_param[cluster][cur_fold]]
        best_mean[cluster][cur_fold] = mean_r[cluster][cur_fold][best_param[cluster][cur_fold]]

In [72]:
np.save("../code/baseline/result/sc-non-disc-cluster-param.npy", best_param)

In [73]:
pd.DataFrame(best_mean).mean().mean()

152.27103606759664

In [74]:
cluster_mean = {}
for cluster in [1, 2, 3]:
    print(pd.DataFrame(best_result[cluster]).mean(axis=1))
    cluster_mean[cluster] = pd.DataFrame(best_result[cluster]).mean(axis=1)

dr         87.096466
dw         22.488325
fridge    366.298752
hvac       70.787800
mw         67.008812
dtype: float64
dr         75.625421
dw         41.967362
fridge    242.082469
hvac      135.496361
mw         69.242489
dtype: float64
dr        144.320653
dw         58.712333
fridge    726.248212
hvac       59.643763
mw        117.046324
dtype: float64


In [75]:
pd.DataFrame(cluster_mean).mean(axis=1)

dr        102.347513
dw         41.056007
fridge    444.876477
hvac       88.642641
mw         84.432542
dtype: float64

# sc with discriminative

In [76]:
dataset = 2
r = {}
mean_r = {}
pred = {}
for cluster in [1, 2 ,3]:
    r[cluster] = {}
    mean_r[cluster] = {}
    pred[cluster] = {}
    for cur_fold in range(5):
        r[cluster][cur_fold] = {}
        mean_r[cluster][cur_fold] = {}
        pred[cluster][cur_fold] = {}
        for num_latent in range(1, 51):
            r[cluster][cur_fold][num_latent] = {}
            mean_r[cluster][cur_fold][num_latent] = {}
            pred[cluster][cur_fold][num_latent] = {}
            for iters in range(10, 110, 10):
                pred[cluster][cur_fold][num_latent][iters] = np.load("../code/baseline/sc-with-nested-cluster/sc-with-pred-{}-{}-{}-{}-{}.npy".format(dataset, cluster, cur_fold, num_latent, iters))
                r[cluster][cur_fold][num_latent][iters] = np.load("../code/baseline/sc-with-nested-cluster/sc-with-error-{}-{}-{}-{}-{}.npy".format(dataset, cluster, cur_fold, num_latent, iters)).item()
                mean_r[cluster][cur_fold][num_latent][iters] = pd.Series(r[cluster][cur_fold][num_latent][iters]).mean()

In [77]:
r[3][0][47][20]

{'dr': 1555.3115159500551,
 'dw': 20.857781477722927,
 'fridge': 102.25225381907963,
 'hvac': 1632.7188922931341,
 'mw': 44.303140512256675}

In [78]:
best_result = {}
best_param = {}
best_mean = {}
best_pred = {}
for cluster in [1, 2, 3]:
    best_result[cluster] = {}
    best_mean[cluster] = {}
    best_param[cluster] = {}
    best_pred[cluster] = {}
    for cur_fold in range(5):
        best_result[cluster][cur_fold] = {} 
        best_param[cluster][cur_fold] = {}
        best_pred[cluster][cur_fold] = {}

In [79]:
pd.DataFrame(mean_r[3][0]).idxmin(axis=1).loc[10]

33

In [83]:
r[3][0][32][80]

{'dr': 109.92697138647574,
 'dw': 20.857781477722927,
 'fridge': 102.25225381907963,
 'hvac': 11.127015695922447,
 'mw': 44.303140512256675}

In [81]:
for cluster in [1,2,3]:
    for cur_fold in range(5):
        
        #find the best
#         temp = pd.DataFrame(mean_r[cluster][cur_fold]).idxmin(axis=1)
        best_error = np.Inf
        for latent in range(1, 51):
            for iters in range(10, 110, 10):
                error = mean_r[cluster][cur_fold][latent][iters]
                if error < best_error:
                    best_error = error
                    best_iters = iters
                    best_latent = latent

            best_param[cluster][cur_fold]['num_latent'] = best_latent
            best_param[cluster][cur_fold]['iters'] = best_iters

            best_result[cluster][cur_fold] = r[cluster][cur_fold][best_latent][best_iters]
            best_mean[cluster][cur_fold] = mean_r[cluster][cur_fold][best_latent][best_iters]
            best_pred[cluster][cur_fold] = pred[cluster][cur_fold][best_latent][best_iters]

In [84]:
best_param[3][0]

{'iters': 80, 'num_latent': 32}

In [85]:
np.save("../code/baseline/result/sc-with-disc-cluster-param.npy", best_param)

In [63]:
for cluster in [1 ,2 ,3]:
    for cur_fold in range(5):
        num_latent = best_param[cluster][cur_fold]['num_latent']
        iters = best_param[cluster][cur_fold]['iters']
        print("python baseline-sc-with-disc-nested-cluster.py 2 {} {} {} {}".format(cluster, cur_fold, num_latent, iters))

python baseline-sc-with-disc-nested-cluster.py 2 1 0 8 90
python baseline-sc-with-disc-nested-cluster.py 2 1 1 3 10
python baseline-sc-with-disc-nested-cluster.py 2 1 2 12 90
python baseline-sc-with-disc-nested-cluster.py 2 1 3 9 90
python baseline-sc-with-disc-nested-cluster.py 2 1 4 1 20
python baseline-sc-with-disc-nested-cluster.py 2 2 0 1 10
python baseline-sc-with-disc-nested-cluster.py 2 2 1 1 10
python baseline-sc-with-disc-nested-cluster.py 2 2 2 1 10
python baseline-sc-with-disc-nested-cluster.py 2 2 3 1 10
python baseline-sc-with-disc-nested-cluster.py 2 2 4 1 10
python baseline-sc-with-disc-nested-cluster.py 2 3 0 26 100
python baseline-sc-with-disc-nested-cluster.py 2 3 1 38 10
python baseline-sc-with-disc-nested-cluster.py 2 3 2 35 10
python baseline-sc-with-disc-nested-cluster.py 2 3 3 34 80
python baseline-sc-with-disc-nested-cluster.py 2 3 4 34 80


In [172]:
np.save("../code/baseline/sc-with-cluster-pred.npy", best_pred)

In [61]:
cluster_mean = {}
for cluster in [1, 2, 3]:
    print(pd.DataFrame(best_result[cluster]).mean(axis=1))
    cluster_mean[cluster] = pd.DataFrame(best_result[cluster]).mean(axis=1)

dr        73.177020
dw        23.414058
fridge    98.244758
hvac       5.693671
mw        20.071766
dtype: float64
dr        77.722514
dw        17.675376
fridge    87.732885
hvac      74.907578
mw         7.890923
dtype: float64
dr        79.832388
dw        16.130939
fridge    93.112450
hvac       7.734536
mw        35.502404
dtype: float64


In [142]:
pd.DataFrame(cluster_mean).mean(axis=1)

dr        76.910641
dw        19.073458
fridge    93.030031
hvac      29.424778
mw        21.155031
dtype: float64

# stf

In [86]:
dataset = 2
r = {}
mean_r = {}
for cluster in [1, 2 ,3]:
    r[cluster] = {}
    mean_r[cluster] = {}
    for cur_fold in range(5):
        r[cluster][cur_fold] = {}
        mean_r[cluster][cur_fold] = {}
        for num_latent in range(1, 21):
            r[cluster][cur_fold][num_latent] = {}
            mean_r[cluster][cur_fold][num_latent] = {}
            for lr in [0.01, 0.1 ,1 ,2]:
                lr = float(lr)
                r[cluster][cur_fold][num_latent][lr] = {}
                mean_r[cluster][cur_fold][num_latent][lr] = {}
                for iters in range(100, 2600, 400):
                    r[cluster][cur_fold][num_latent][lr][iters] = np.load("../code/baseline/stf-cluster/valid/stf-error-{}-{}-{}-{}-{}-{}.npy".format(dataset, cluster, cur_fold, num_latent, lr, iters)).item()
                    mean_r[cluster][cur_fold][num_latent][lr][iters] = pd.Series(r[cluster][cur_fold][num_latent][lr][iters]).mean()

In [87]:
best_result = {}
best_param = {}
best_mean = {}
for cluster in [1, 2, 3]:
    best_result[cluster] = {}
    best_mean[cluster] = {}
    best_param[cluster] = {}
    for cur_fold in range(5):
        best_result[cluster][cur_fold] = {} 
        best_param[cluster][cur_fold] = {}

In [88]:
for cluster in [1, 2, 3]:
    for cur_fold in range(5):
        
        #find the best
        best_error = np.Inf
        for num_latent in range(1, 21):
            for lr in [0.01, 0.1, 1, 2]:
                lr = float(lr)
                for iters in range(100, 2600, 400):
                    error = mean_r[cluster][cur_fold][num_latent][lr][iters]
                    if error < best_error:
                        best_error = error
                        best_lr = lr
                        best_iters = iters
                        best_latent = num_latent
        
        best_param[cluster][cur_fold]['num_latent'] = best_latent
        best_param[cluster][cur_fold]['iters'] = best_iters
        best_param[cluster][cur_fold]['lr'] = best_lr


        
        best_result[cluster][cur_fold] = r[cluster][cur_fold][best_latent][best_lr][best_iters]
        best_mean[cluster][cur_fold] = mean_r[cluster][cur_fold][best_latent][best_lr][best_iters]

In [89]:
np.save("../code/baseline/result/stf-cluster-param.npy", best_param)

In [92]:
for cluster in [1, 2, 3]:
    for cur_fold in range(5):
        num_latent = best_param[cluster][cur_fold]['num_latent'] 
        iters = best_param[cluster][cur_fold]['iters']
        lr = best_param[cluster][cur_fold]['lr']
        print("python baseline-stf-nested-cluster.py 2 {} {} {} {} {} &".format(cluster, cur_fold, num_latent, lr, iters))

python baseline-stf-nested-cluster.py 2 1 0 18 1.0 2500 &
python baseline-stf-nested-cluster.py 2 1 1 4 2.0 2500 &
python baseline-stf-nested-cluster.py 2 1 2 18 2.0 900 &
python baseline-stf-nested-cluster.py 2 1 3 7 2.0 100 &
python baseline-stf-nested-cluster.py 2 1 4 3 1.0 900 &
python baseline-stf-nested-cluster.py 2 2 0 16 2.0 500 &
python baseline-stf-nested-cluster.py 2 2 1 16 0.1 1300 &
python baseline-stf-nested-cluster.py 2 2 2 13 2.0 500 &
python baseline-stf-nested-cluster.py 2 2 3 19 2.0 1300 &
python baseline-stf-nested-cluster.py 2 2 4 16 2.0 500 &
python baseline-stf-nested-cluster.py 2 3 0 15 1.0 2500 &
python baseline-stf-nested-cluster.py 2 3 1 16 2.0 2100 &
python baseline-stf-nested-cluster.py 2 3 2 19 1.0 100 &
python baseline-stf-nested-cluster.py 2 3 3 15 1.0 500 &
python baseline-stf-nested-cluster.py 2 3 4 16 2.0 500 &


In [93]:
cluster_mean = {}
for cluster in [1, 2, 3]:
    print(pd.DataFrame(best_result[cluster]).mean(axis=1))
    cluster_mean[cluster] = pd.DataFrame(best_result[cluster]).mean(axis=1)

dr        118.363768
dw         24.475190
fridge     46.830976
hvac      492.061653
mw         10.672978
dtype: float64
dr         87.139380
dw         14.567767
fridge     30.509814
hvac      307.187959
mw          6.563174
dtype: float64
dr        179.563214
dw         50.981014
fridge     54.738440
hvac      712.901891
mw         35.726113
dtype: float64


In [94]:
pd.DataFrame(cluster_mean).mean(axis=1)

dr        128.355454
dw         30.007990
fridge     44.026410
hvac      504.050501
mw         17.654088
dtype: float64

# mtf

In [101]:
dataset = 2
r = {}
mean_r = {}
for cluster in [1, 2 ,3]:
    r[cluster] = {}
    mean_r[cluster] = {}
    for cur_fold in range(5):
        r[cluster][cur_fold] = {}
        mean_r[cluster][cur_fold] = {}
        for num_latent in range(1, 21):
            r[cluster][cur_fold][num_latent] = {}
            mean_r[cluster][cur_fold][num_latent] = {}
            for lr in [0.01, 0.1 ,1 ,2]:
                lr = float(lr)
                r[cluster][cur_fold][num_latent][lr] = {}
                mean_r[cluster][cur_fold][num_latent][lr] = {}
                for iters in range(100, 2600, 400):
                    r[cluster][cur_fold][num_latent][lr][iters] = np.load("../code/baseline/mtf-cluster/valid/mtf-error-{}-{}-{}-{}-{}-{}.npy".format(dataset, cluster, cur_fold, num_latent, lr, iters)).item()
                    mean_r[cluster][cur_fold][num_latent][lr][iters] = pd.Series(r[cluster][cur_fold][num_latent][lr][iters]).mean()

In [102]:
best_result = {}
best_param = {}
best_mean = {}
for cluster in [1, 2, 3]:
    best_result[cluster] = {}
    best_mean[cluster] = {}
    best_param[cluster] = {}
    for cur_fold in range(5):
        best_result[cluster][cur_fold] = {} 
        best_param[cluster][cur_fold] = {}

In [103]:
for cluster in [1, 2, 3]:
    for cur_fold in range(5):
        
        #find the best
        best_error = np.Inf
        for num_latent in range(1, 21):
            for lr in [0.01, 0.1, 1, 2]:
                lr = float(lr)
                for iters in range(100, 2600, 400):
                    error = mean_r[cluster][cur_fold][num_latent][lr][iters]
                    if error < best_error:
                        best_error = error
                        best_lr = lr
                        best_iters = iters
                        best_latent = num_latent
        
        best_param[cluster][cur_fold]['num_latent'] = best_latent
        best_param[cluster][cur_fold]['iters'] = best_iters
        best_param[cluster][cur_fold]['lr'] = best_lr


        
        best_result[cluster][cur_fold] = r[cluster][cur_fold][best_latent][best_lr][best_iters]
        best_mean[cluster][cur_fold] = mean_r[cluster][cur_fold][best_latent][best_lr][best_iters]

In [104]:
np.save("../code/baseline/result/mtf-cluster-param.npy", best_param)

In [105]:
for cluster in [1, 2, 3]:
    for cur_fold in range(5):
        num_latent = best_param[cluster][cur_fold]['num_latent'] 
        iters = best_param[cluster][cur_fold]['iters']
        lr = best_param[cluster][cur_fold]['lr']
        print("python baseline-mtf-nested-cluster.py 2 {} {} {} {} {}&".format(cluster, cur_fold, num_latent, lr, iters))

python baseline-mtf-nested-cluster.py 2 1 0 4 2.0 100&
python baseline-mtf-nested-cluster.py 2 1 1 20 2.0 500&
python baseline-mtf-nested-cluster.py 2 1 2 2 1.0 100&
python baseline-mtf-nested-cluster.py 2 1 3 2 1.0 100&
python baseline-mtf-nested-cluster.py 2 1 4 17 2.0 2500&
python baseline-mtf-nested-cluster.py 2 2 0 11 2.0 2500&
python baseline-mtf-nested-cluster.py 2 2 1 12 0.1 2500&
python baseline-mtf-nested-cluster.py 2 2 2 18 1.0 2500&
python baseline-mtf-nested-cluster.py 2 2 3 18 0.1 2500&
python baseline-mtf-nested-cluster.py 2 2 4 16 2.0 900&
python baseline-mtf-nested-cluster.py 2 3 0 15 1.0 1300&
python baseline-mtf-nested-cluster.py 2 3 1 16 2.0 500&
python baseline-mtf-nested-cluster.py 2 3 2 9 1.0 100&
python baseline-mtf-nested-cluster.py 2 3 3 16 2.0 100&
python baseline-mtf-nested-cluster.py 2 3 4 16 2.0 100&


In [123]:
cluster_mean = {}
for cluster in [1, 2, 3]:
    print(pd.DataFrame(best_result[cluster]).mean(axis=1))
    cluster_mean[cluster] = pd.DataFrame(best_result[cluster]).mean(axis=1)

dr        114.125645
dw         28.018820
fridge     39.003502
hvac      372.947448
mw         18.795039
dtype: float64
dr        110.483846
dw         25.713758
fridge     36.586460
hvac      357.318197
mw          7.876719
dtype: float64
dr        153.566914
dw         58.198180
fridge     36.394793
hvac      566.114716
mw         34.218185
dtype: float64


In [127]:
pd.DataFrame(cluster_mean).mean(axis=1)

dr        126.058802
dw         37.310253
fridge     37.328252
hvac      432.126787
mw         20.296647
dtype: float64